In [1]:
import numpy as np 
import os 
import pandas as pd

reflect = 30
tuple_a = (10, 20, 10, 10)
tuple_b = (25, 27, 2, 5)
tuple_c = (30, 40, 10, 3)

path = "../GEO3000/code/data/discharge_data_100_cleaned/"

def get_files(folder_path):
        folder_list_data = []
        folder_list_supplementary = []
        file_type = ".q"

        folder = os.scandir(folder_path)               # Reads the directory
        for file in folder:                                         # Loop over files
            if file.name[-len(file_type):] == file_type:  # Checks for files of filetype
                folder_list_data.append(file.name)             # Adds filenames with correct filetype to list
            else:
                folder_list_supplementary.append(file.name)    # Add all other files to a supplementary list
        folder.close()      
        return folder_list_data

files = get_files(path)

def read_file(folder_list_data, path):
    a = True
    return_list = []
    for file in folder_list_data:
        nan_list = []
        file_object = open(path + file, "r")
        file_object.readline()
        for line in file_object:
            if line[:2] == "#*":
                nan_list.append((0,0,0,0))
                if a:
                    df_output = pd.read_csv(file_object, 
                                            delim_whitespace=True, 
                                            header=0, 
                                            index_col=[0,1,2], 
                                        )
                    a = False
                    print(nan_list)
                break
            nan_list.append(eval(line[1:]))
        file_object.close()
                
        return_list.append(nan_list)
    return return_list, df_output

nan_list, df_raw = read_file(files, path)

# hyearstart = (int(df.iloc[0].name[0]), 9, 1)
# hyearend = (int(df.iloc[0].name[0] + 1), 8, 31)
# tmp_nan_list = [tuple_a, tuple_b, tuple_c]#, tuple_d].groupby(,axis=0)

[(0, 0, 0, 0)]


In [2]:
df = df_raw.copy() # I can still tuple index this. 
df

discharge
year month day           
1991 9     1     0.083963
           2     0.083963
           3     0.083963
           4     0.083963
           5     0.070818
...                   ...
2020 8     27    0.178247
           28    0.127344
           29    0.121626
           30    0.111551
           31    0.100905

[10593 rows x 1 columns]

In [3]:
df.groupby(by='year').mean()


def assign_wy(row):
    if row.name[1]>8:
        return(int(row.name[0] + 1))
    else:
        return(int(row.name[0]))
df['WY'] = df.apply(lambda x: assign_wy(x), axis=1)
df = df[["WY", "discharge"]]
df_2 = df.copy()
df_2

WY  discharge
year month day                 
1991 9     1    1992   0.083963
           2    1992   0.083963
           3    1992   0.083963
           4    1992   0.083963
           5    1992   0.070818
...              ...        ...
2020 8     27   2020   0.178247
           28   2020   0.127344
           29   2020   0.121626
           30   2020   0.111551
           31   2020   0.100905

[10593 rows x 2 columns]

In [14]:
def BFI(Q, alpha=0.925, passes=3, ReturnQbase=False, reflect=30):
    #Q = pd.DataFrame(Q, columns=["dis"]) # Adding input to DF
    Q_input_raw = Q.to_numpy()
    # Above is not needed in actual program
    def first_pass(Q, a):
        b = 0.5 * (1+a)
        Qf1 = np.zeros(len(Q)) # Empty quickflow 
        Qf1[0] = Q[0]
        for i in range(1, len(Qf1)):
            Qf1[i] = a*Qf1[i-1] + b*(Q[i]-Q[i-1])
            
        Qb1 = np.where(Qf1 > 0, Q-Qf1, Q)
        
        return pd.DataFrame({"Qf": Qf1, "Qb": Qb1})
    
    def backwards_pass(Q, a):
        n = len(Q["Qb"])
        Qb = Q["Qb"]
        b = 0.5 * (1+a)
        
        Qf2 = np.zeros(n) # Empty array
        Qf2[-1] = Qb.iloc[-1]
        
        for i in range(n-2, 0, -1):
            Qf2[i] = a*Qf2[i+1] + b*(Qb.iloc[i] - Qb.iloc[i+1])
            
        Qb2 = np.where(Qf2 > 0, Qb - Qf2, Qb)
        
        return pd.DataFrame({"Qf": Qf2, "Qb": Qb2})
    
    def forward_pass(Q, a):
        n = len(Q["Qb"])
        Qb = Q["Qb"]
        b = 0.5 * (1+a)
        
        Qf2 = np.zeros(n) # Empty array
        Qf2[0] = Qb.iloc[0]
        
        for i in range(1, n):
            Qf2[i] = a*Qf2[i-1] + b*(Qb.iloc[i] - Qb.iloc[i-1])
            
        Qb2 = np.where(Qf2 > 0, Qb - Qf2, Qb)
        
        return pd.DataFrame({"Qf": Qf2, "Qb": Qb2})
    
    
    Qin = np.pad(Q_input_raw, (reflect, reflect), mode="reflect")
        
        
    df1 = first_pass(Qin, alpha)
    df2 = backwards_pass(df1, alpha)
    df3 = forward_pass(df2, alpha)
    
    return df3["Qb"][reflect:-reflect].to_numpy()


In [15]:
# Run the smoothing on the entire dataset. 
qBaseflow = BFI(df["discharge"])

In [16]:
qBaseflow

array([0.07945325, 0.07979148, 0.08010435, ..., 0.121626  , 0.111551  ,
       0.100905  ])

In [17]:
df_2 = df.copy()
df_2["Qb"] = qBaseflow
df_2

WY  discharge        Qb
year month day                           
1991 9     1    1992   0.083963  0.079453
           2    1992   0.083963  0.079791
           3    1992   0.083963  0.080104
           4    1992   0.083963  0.080394
           5    1992   0.070818  0.070818
...              ...        ...       ...
2020 8     27   2020   0.178247  0.178247
           28   2020   0.127344  0.127344
           29   2020   0.121626  0.121626
           30   2020   0.111551  0.111551
           31   2020   0.100905  0.100905

[10593 rows x 3 columns]

In [21]:
df_3 = df_2.groupby("WY").sum()
df_3["BFI"] = df_3["Qb"]/df_3["discharge"]
df_3

,discharge,Qb,BFI
WY,,,
1992,425.407410,206.474997,0.485358
1993,755.781031,324.149765,0.428894
1994,1091.735770,465.694344,0.426563
1995,1131.482239,572.260489,0.505762
1996,648.402967,289.796945,0.446940
1997,754.004539,353.658177,0.469040
1998,891.931979,406.944789,0.456251
1999,1253.936941,604.154557,0.481806
2000,844.767363,384.139456,0.454728


In [9]:
for x,y in zip(files, nan_list):
    print(x,y)
    print()

2.634.q [(0, 0, 0, 0)]

12.209.q [(7875, 7876, 1, 7875), (0, 0, 0, 0)]

105.1.q [(0, 0, 0, 0)]

88.4.q [(0, 0, 0, 0)]

2.28.q [(0, 0, 0, 0)]

173.8.q [(7792, 7948, 156, 7792), (10194, 10240, 46, 2246), (12313, 12809, 496, 2073), (13038, 13385, 347, 229), (15462, 15588, 126, 2077), (15770, 15894, 124, 182), (16757, 16813, 56, 863), (0, 0, 0, 0)]

2.265.q [(9253, 10012, 759, 9253), (10013, 10029, 16, 1), (10030, 10082, 52, 1), (10083, 10112, 29, 1), (10113, 10171, 58, 1), (10172, 10175, 3, 1), (10176, 10189, 13, 1), (10190, 10218, 28, 1), (10219, 10258, 39, 1), (10259, 10281, 22, 1), (10282, 10349, 67, 1), (10714, 10896, 182, 365), (10897, 10899, 2, 1), (10900, 10901, 1, 1), (10902, 10903, 1, 1), (10904, 10905, 1, 1), (10906, 10908, 2, 1), (10909, 10910, 1, 1), (10911, 10913, 2, 1), (10914, 10915, 1, 1), (10916, 10917, 1, 1), (10918, 10919, 1, 1), (10920, 10921, 1, 1), (10922, 10923, 1, 1), (10924, 10925, 1, 1), (10926, 11050, 124, 1), (11051, 11057, 6, 1), (11058, 11064, 6, 1), (11065, 

In [10]:
df.index

MultiIndex([(1991, 9,  1),
            (1991, 9,  2),
            (1991, 9,  3),
            (1991, 9,  4),
            (1991, 9,  5),
            (1991, 9,  6),
            (1991, 9,  7),
            (1991, 9,  8),
            (1991, 9,  9),
            (1991, 9, 10),
            ...
            (2020, 8, 22),
            (2020, 8, 23),
            (2020, 8, 24),
            (2020, 8, 25),
            (2020, 8, 26),
            (2020, 8, 27),
            (2020, 8, 28),
            (2020, 8, 29),
            (2020, 8, 30),
            (2020, 8, 31)],
           names=['year', 'month', 'day'], length=10593)

In [11]:
tmp_list = []
tmp_list2 = []
tmp_list3 = []
tmp_list4 = []

for nan_indicies in tmp_nan_list:
    v1 = nan_indicies[0]
    v2 = nan_indicies[1]
    v3 = nan_indicies[2]
    v4 = nan_indicies[3]
    if v4 < reflect:
        # If this is the case we want to skip all the series which are shorter than the reflect length.
        # Would then use the previous initial value and last value as the edges of the nan-series.

        # This method works but is dumb af
        tmp_list.append(v1)  # Append first value
        tmp_list.append(v2)  # Append last value
        tmp_list2.append(v3)  # Append Length_Nan
        tmp_list3.append(v4)  # Append Length_NoNan
    elif nan_indicies == tmp_nan_list[-1]: 
        a = tmp_list[0]
        b = tmp_list[-1]
        tmp_list4.append((a,b,b-a,tmp_list3[0]))

    else:
        a = tmp_list[0]
        b = tmp_list[-1]
        c = b - a

        tmp_list4.append((a, b, c, tmp_list3[0]))

        tmp_list = []
        tmp_list2 = []
        tmp_list3 = []
tmp_list4

NameError: name 'tmp_nan_list' is not defined